# Twitter sentiment analysis using RoBERTa model

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

tweet = "@eshwarnadh today's cold @ home 😒 https://eshwar.com"
# tweet = 'Great content! subscribed 😉'

# precprcess tweet
tweet_words = []

for word in tweet.split(' '):
    if word.startswith('@') and len(word) > 1:
        word = '@user'
    
    elif word.startswith('http'):
        word = "http"
    tweet_words.append(word)

tweet_proc = " ".join(tweet_words)

## load model and tokenizer
roberta = "cardiffnlp/twitter-roberta-base-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)
labels = ['Negative', 'Neutral', 'Positive'] 

# # sentiment analysis
encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')
print(encoded_tweet)
output = model(encoded_tweet['input_ids'], encoded_tweet['attention_mask'])
print(output)
# output = model(**encoded_tweet) another way to write above line

scores = output[0][0].detach().numpy()
scores = softmax(scores)

for i in range(len(scores)):
    
    l = labels[i]
    s = scores[i]
    print(l,s)




{'input_ids': tensor([[    0,  1039, 12105,   452,    18,  2569,   787,   184, 17841, 10659,
          2054,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
SequenceClassifierOutput(loss=None, logits=tensor([[ 1.3710,  0.3350, -1.7215]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
Negative 0.7141535
Neutral 0.25342986
Positive 0.032416593


In [2]:
pip install gradio

Note: you may need to restart the kernel to use updated packages.


In [3]:
import gradio as gr

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
def classify_tweet(tweet):
    tweet_words = []

    for word in tweet.split(' '):
        if word.startswith('@') and len(word) > 1:
            word = '@user'

        elif word.startswith('http'):
            word = "http"
        tweet_words.append(word)

    tweet_proc = " ".join(tweet_words)
    roberta = "cardiffnlp/twitter-roberta-base-sentiment"
    model = AutoModelForSequenceClassification.from_pretrained(roberta)
    tokenizer = AutoTokenizer.from_pretrained(roberta)
    labels = ['Negative', 'Neutral', 'Positive'] 

    # # sentiment analysis
    encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')
    output = model(encoded_tweet['input_ids'], encoded_tweet['attention_mask'])
    # output = model(**encoded_tweet) another way to write above line

    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    predictions = {labels[i]:float(scores[i])  for i in range(len(scores))}
#     for i in range(len(scores)):
#         l = labels[i]
#         s = scores[i]
#         predictions[l] = s
    return predictions

In [5]:
interface=gr.Interface(
            fn=classify_tweet,
            inputs="textbox",
            outputs=gr.outputs.Label(num_top_classes=3))
interface.launch()

C:\Users\eswar\anaconda3\lib\site-packages\gradio\outputs.py:197: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
C:\Users\eswar\anaconda3\lib\site-packages\gradio\outputs.py:200: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  super().__init__(num_top_classes=num_top_classes, type=type, label=label)


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
